In [ ]:
import sys
sys.path.insert(0, '/nfs/general/shared/code/rpi_color_detector/classifier/')
import image_provider
import tensorflow as tf

In [13]:
def TrainingDataGen():
    ip = image_provider.ImageProvider()
    d = ip.listImages('classified')
    validClassNames = sorted(
        [x for x in d if x not in ['unclassified', '_garbage']])
    print(validClassNames)
    for i, className in enumerate(validClassNames):
        for image_path in d[className]:
            yield ip.filePath(image_path), i

def LoadImage(filename):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    image.set_shape([480, 640, 3])
    return image

def ExtractRoiForPrediction(image):
    x=0.3
    target_width=int(640*x)
    target_height=int(480*x)
    offset_width=(640-target_width)//2
    offset_height=(480-target_height)//2
    return tf.image.crop_to_bounding_box(image, offset_height, offset_width, target_height, target_width)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomTranslation(0.05, 0.05)
    ]
)

AUTOTUNE = tf.data.AUTOTUNE

training_data = tf.data.Dataset.from_generator(TrainingDataGen, output_signature=(
    tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.int32)))
training_data = training_data.map(lambda imgName, label : (LoadImage(imgName), label), deterministic=False, num_parallel_calls=AUTOTUNE)
training_data = training_data.cache()
training_data = training_data.repeat()
training_data = training_data.shuffle(buffer_size=1000)
training_data = training_data.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
training_data = training_data.map(lambda x, y: (ExtractRoiForPrediction(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
training_data = training_data.batch(64)
training_data = training_data.prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (5, 5), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7)
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
# starting_point = tf.saved_model.load('/nfs/general/shared/tf_models/color_classifier')

# print(starting_point.variables)

# print(model.variables)

tb_callback = tf.keras.callbacks.TensorBoard(
    '/tmp/logs', update_freq=1, histogram_freq=1, write_images=True)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)

model.fit(training_data, epochs=1200, steps_per_epoch=32, callbacks=[tb_callback, model_checkpoint_callback])

tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier3')
converter = tf.lite.TFLiteConverter.from_saved_model('/nfs/general/shared/tf_models/color_classifier') # path to the SavedModel directory
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)

Epoch 1/1200
['black', 'blue', 'green', 'pink', 'salmon', 'white', 'yellow']
32/32 [==============================] - 2s 53ms/step - loss: 2.8595 - accuracy: 0.2168
Epoch 2/1200
32/32 [==============================] - 2s 49ms/step - loss: 1.9358 - accuracy: 0.2251
Epoch 3/1200
32/32 [==============================] - 1s 46ms/step - loss: 1.8524 - accuracy: 0.2285
Epoch 4/1200
32/32 [==============================] - 1s 47ms/step - loss: 1.9028 - accuracy: 0.2271
Epoch 5/1200
32/32 [==============================] - 1s 44ms/step - loss: 1.6395 - accuracy: 0.2920
Epoch 6/1200
32/32 [==============================] - 1s 43ms/step - loss: 1.8091 - accuracy: 0.2817
Epoch 7/1200
32/32 [==============================] - 1s 47ms/step - loss: 1.4092 - accuracy: 0.4570
Epoch 8/1200
32/32 [==============================] - 1s 46ms/step - loss: 1.6751 - accuracy: 0.3184
Epoch 9/1200
32/32 [==============================] - 1s 47ms/step - loss: 1.2649 - accuracy: 0.4873
Epoch 10/1200
32/32 [======

2022-04-23 11:48:13.085320: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /nfs/general/shared/tf_models/color_classifier3/assets


2022-04-23 11:48:13.810357: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-23 11:48:13.810384: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-23 11:48:13.810983: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /nfs/general/shared/tf_models/color_classifier
2022-04-23 11:48:13.813011: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-23 11:48:13.813022: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /nfs/general/shared/tf_models/color_classifier
2022-04-23 11:48:13.817289: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-23 11:48:13.859280: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /nfs/general/shared/tf_models/color_classifier
2022-04-23 11:48:13.874145: I tensorflow/cc/saved_

232332

In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model('/nfs/general/shared/tf_models/color_classifier3') # path to the SavedModel directory
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)

2022-04-23 11:48:45.971891: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-23 11:48:45.971916: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-23 11:48:45.972107: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /nfs/general/shared/tf_models/color_classifier3
2022-04-23 11:48:45.973967: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-23 11:48:45.973977: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /nfs/general/shared/tf_models/color_classifier3
2022-04-23 11:48:45.977614: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-23 11:48:46.019558: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /nfs/general/shared/tf_models/color_classifier3
2022-04-23 11:48:46.032498: I tensorflow/cc/sav

232412